In [1]:
import collections
import copy
import datetime
import itertools
import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

AttributeError: 'builtin_function_or_method' object has no attribute 'glob'

In [2]:
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay

In [4]:
cbe01 = False
for Mu in G.matchups:
    print(Mu)
    if Mu.tournament.Id == "cbe01":
        cbe01 = True
    if not cbe01:
        continue
    if not hasattr(Mu, "configmaker"):
        continue
    CM = Mu.configmaker(direct=True, refresh_cache=False)
    CM.Ma = Mu.match
    CM.PlayerRetirements()
    print()

Matchup('cibbl', '41656', 0, 854257, 854500)
Matchup('cibbl', '41656', 0, 854500, 855120)
Matchup('cibbl', '41656', 0, 854507, 854752)
Matchup('cibbl', '41656', 0, 855072, 855228)
Matchup('cibbl', '41656', 0, 854507, 854678)
Matchup('cibbl', '41656', 0, 854690, 855072)
Matchup('cibbl', '41656', 0, 854600, 854678)
Matchup('cibbl', '41656', 0, 854507, 854690)
Matchup('cibbl', '41656', 0, 854502, 855072)
Matchup('cibbl', '41656', 0, 854480, 854600)
Matchup('cibbl', '41656', 0, 854600, 854690)
Matchup('cibbl', '41656', 0, 854257, 854502)
Matchup('cibbl', '41656', 0, 854257, 855530)
Matchup('cibbl', '41656', 0, 854678, 855530)
Matchup('cibbl', '41656', 0, 854752, 855530)
Matchup('cibbl', '41656', 0, 854500, 855753)
Matchup('cibbl', '41656', 0, 855120, 855753)
Matchup('cibbl', '41656', 0, 855228, 855753)
Matchup('cibbl', '41656', 0, 855228, 856203)
Matchup('cibbl', '41656', 0, 854752, 856372)
Matchup('cibbl', '41656', 0, 856203, 856372)
Matchup('cibbl', '41741', 1, 855530, 856372)
Matchup('c

on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/01-0873361-0874602.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/01-0873361-0874602.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/01-0873361-0874602.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/01-0873361-0874602.json') True

Matchup('cibbl', '43152', 1, 873349, 874545)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043152/01-0873349-0874545.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043152/01-0873349-0874545.json') True

AbstractMatchup('cibbl', 'cbe01', 1, 873349, 874545)
Matchup('cibbl', '43150', 1, 876711, 876774)

Matchup('cibbl', '43153', 1, 854257, 870410)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043153/01-0854257-0870410.json') True

AbstractMatchup('cibbl', 'cbe01', 1, 872044, 874452)

on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043149/03-0879779-0880296.json') True

Matchup('cibbl', '43150', 2, 876711, 876969)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043150/02-0876711-0876969.json') True

Matchup('cibbl', '43152', 3, 872044, 874545)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043152/03-0872044-0874545.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043152/03-0872044-0874545.json') True

AbstractMatchup('cibbl', 'cbe01', 3, 872044, 874545)
Matchup('cibbl', '43150', 2, 873369, 879790)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043150/02-0873369-0879790.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043150/02-0873369-0879790.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043150/02-0873369-0879790.json') True

Matchup('cibbl', '43154', 2, 855753, 872475)
on chan

on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/03-0854507-0855120.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/03-0854507-0855120.json') True

AbstractMatchup('cibbl', 'cbe01', 3, 854507, 855120)
Matchup('cibbl', '43151', 3, 865063, 874602)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/03-0865063-0874602.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/03-0865063-0874602.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/03-0865063-0874602.json') True

AbstractMatchup('cibbl', 'cbe01', 3, 865063, 874602)
AbstractMatchup('cibbl', 'cbe01', 4, 874452, 874545)
Matchup('cibbl', '43152', 4, 874452, 874545)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043152/04-0874452-0874545.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043152/04-0874452-0874545.json'


Matchup('cibbl', '43154', 5, 867862, 872475)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043154/05-0867862-0872475.json') True

Matchup('cibbl', '43151', 5, 855120, 869156)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/05-0855120-0869156.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043151/05-0855120-0869156.json') True

AbstractMatchup('cibbl', 'cbe01', 5, 855120, 869156)
Matchup('cibbl', '43154', 5, 873945, 874302)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043154/05-0873945-0874302.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043154/05-0873945-0874302.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043154/05-0873945-0874302.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00043154/05-0873945-0874302.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data

...


Matchup('cibbl', '46706', 1, 924405, 928360)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046706/01-0924405-0928360.json') True

Matchup('cibbl', '46707', 1, 906259, 916031)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/01-0906259-0916031.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/01-0906259-0916031.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/01-0906259-0916031.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/01-0906259-0916031.json') True

Matchup('cibbl', '46707', 1, 908411, 916217)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/01-0908411-0916217.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/01-0908411-0916217.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/01-0908411-0916217.json') Tru


Matchup('cibbl', '46687', 1, 854678, 871787)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046687/01-0854678-0871787.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046687/01-0854678-0871787.json') True

Matchup('cibbl', '46710', 2, 872044, 907600)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/02-0872044-0907600.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/02-0872044-0907600.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/02-0872044-0907600.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/02-0872044-0907600.json') True

Matchup('cibbl', '46709', 1, 854507, 898574)

Matchup('cibbl', '46705', 1, 880378, 886002)

Matchup('cibbl', '46709', 2, 888994, 919838)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046709/02-0888994-0919838.json') True
on change JSON

on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046687/02-0871787-0886114.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046687/02-0871787-0886114.json') True

Matchup('cibbl', '46710', 3, 899682, 922608)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/03-0899682-0922608.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/03-0899682-0922608.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/03-0899682-0922608.json') True

Matchup('cibbl', '46711', 2, 924370, 926596)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046711/02-0924370-0926596.json') True

Matchup('cibbl', '46707', 3, 896134, 916031)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/03-0896134-0916031.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/03-0896134-0916031.json') Tru


Matchup('cibbl', '46688', 4, 874452, 897376)

Matchup('cibbl', '46707', 4, 891237, 908411)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/04-0891237-0908411.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/04-0891237-0908411.json') True

Matchup('cibbl', '46710', 4, 864291, 922608)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/04-0864291-0922608.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046710/04-0864291-0922608.json') True

Matchup('cibbl', '46707', 4, 896134, 906259)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707/04-0896134-0906259.json') True

Matchup('cibbl', '46705', 3, 874302, 886002)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046705/03-0874302-0886002.json') True

Matchup('cibbl', '46707', 4, 897353, 916217)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046707

on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046688/05-0906026-0909966.json') True

Matchup('cibbl', '46711', 4, 926596, 928392)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046711/04-0926596-0928392.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046711/04-0926596-0928392.json') True
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046711/04-0926596-0928392.json') True

Matchup('cibbl', '46687', 5, 874545, 878945)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046687/05-0874545-0878945.json') True

Matchup('cibbl', '46687', 5, 886114, 889232)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046687/05-0886114-0889232.json') True

Matchup('cibbl', '46706', 5, 913702, 924405)
on change JSONFile('/home/adam/FUMBBL/cibblbibbl-data/cibbl/matchup/00046706/05-0913702-0924405.json') True

Matchup('cibbl', '46706', 5, 919662, 923094)
on change JSON

In [3]:
T = self = G.tournaments["44074"]

In [4]:
T.playerperformances()

{NormalPlayer('11936932'): {'td': 0,
  'blocks': 12,
  'turns': 53,
  'int': 0,
  'spp': 0,
  'mvp': 0,
  'prespp': 0,
  'fouls': 0,
  'comp': 0,
  'rush': 8,
  'pass': 1,
  'cas': 0,
  'team': Team(885940),
  'name': 'The Phantom Wanderer',
  'type': 'R',
  'perf': [('L', 3946296),
   ('L', 3947024),
   ('W', 3956129),
   ('L', 3956796),
   ('D', 3959967)]},
 NormalPlayer('11936933'): {'td': 2,
  'blocks': 21,
  'turns': 72,
  'int': 0,
  'spp': 11,
  'mvp': 0,
  'prespp': 53,
  'fouls': 0,
  'comp': 3,
  'rush': 43,
  'pass': 2,
  'cas': 1,
  'team': Team(885940),
  'name': 'The Elegant Phantom',
  'type': 'R',
  'perf': [('L', 3946296),
   ('L', 3947024),
   ('W', 3956129),
   ('L', 3956796),
   ('D', 3959967)]},
 NormalPlayer('11936937'): {'td': 2,
  'blocks': 32,
  'turns': 74,
  'int': 0,
  'spp': 12,
  'mvp': 1,
  'prespp': 45,
  'fouls': 0,
  'comp': 1,
  'rush': 40,
  'pass': 2,
  'cas': 0,
  'team': Team(885940),
  'name': "Revydas 'The Dark Star' Nevar",
  'type': 'R',
  'pe

In [12]:
perfkeys = {
    "blocks",
    "cas",
    "comp",
    "fouls",
    "int",
    "mvp",
    "pass",
    "prespp",
    "rush",
    "spp",
    "td",
    "turns",
}

In [37]:
perf = {}
for Mu in self.matchups:
  if Mu.excluded == "yes":
    continue
  matchId = (Mu.match.Id if Mu.match else None)
  for teamId, d0 in Mu.config["player"].items():
    Te = cibblbibbl.team.Team(int(teamId))
    r = Mu.config["team"][teamId]["r"]
    for playerId, d1 in d0.items():
      Pl = cibblbibbl.player.player(playerId)
      if not (Pl in perf):
        perf[Pl] = {k: d1.get(k, 0) for k in perfkeys}
        perf[Pl]["team"] = Te
        perf[Pl]["name"] = d1["name"]
        perf[Pl]["type"] = d1["type"]
        perf[Pl]["perf"] = [(r, matchId),]
      else:
        for k in perfkeys:
          perf[Pl][k] += d1.get(k, 0)
        perf[Pl]["perf"].append((r, matchId))
      if d1.get("retired") is not None:
        perf[Pl]["retired"] = d1["retired"]
      if d1.get("dead") is not None:
        perf[Pl]["dead"] = copy.copy(d1["dead"]._data)

In [38]:
perf

{NormalPlayer('11936932'): {'comp': 0,
  'fouls': 0,
  'prespp': 0,
  'pass': 1,
  'rush': 8,
  'mvp': 0,
  'blocks': 12,
  'spp': 0,
  'cas': 0,
  'td': 0,
  'turns': 53,
  'int': 0,
  'team': Team(885940),
  'name': 'The Phantom Wanderer',
  'type': 'R',
  'perf': [('L', 3946296),
   ('L', 3947024),
   ('W', 3956129),
   ('L', 3956796),
   ('D', 3959967)]},
 NormalPlayer('11936933'): {'comp': 3,
  'fouls': 0,
  'prespp': 53,
  'pass': 2,
  'rush': 43,
  'mvp': 0,
  'blocks': 21,
  'spp': 11,
  'cas': 1,
  'td': 2,
  'turns': 72,
  'int': 0,
  'team': Team(885940),
  'name': 'The Elegant Phantom',
  'type': 'R',
  'perf': [('L', 3946296),
   ('L', 3947024),
   ('W', 3956129),
   ('L', 3956796),
   ('D', 3959967)]},
 NormalPlayer('11936937'): {'comp': 1,
  'fouls': 0,
  'prespp': 45,
  'pass': 2,
  'rush': 40,
  'mvp': 1,
  'blocks': 32,
  'spp': 12,
  'cas': 0,
  'td': 2,
  'turns': 74,
  'int': 0,
  'team': Team(885940),
  'name': "Revydas 'The Dark Star' Nevar",
  'type': 'R',
  'pe

In [26]:
for k in sorted(perf, key=lambda k: perf[k]["td"], reverse=True):
    print(f'{k.name}, {perf[k]["td"]}')

Shinesa, 10
Darkreaper, 5
Cargavel Stormcrow, 3
Shila Swiftdeath, 3
The Elegant Phantom, 2
Revydas 'The Dark Star' Nevar, 2
Sinthoras Bloodflame, 2
Evilonah Fellblade, 2
Ornthalas Eaglesflight, 2
Lydia, 2
Red Flow, 1
Crimson Flash, 1
Caphalor Darkstar, 1
Malaguth Blackmoon, 1
Rageshadow, 1
Rasith Duskchill, 1
Vaecho Firestarter, 1
Deathfury, 1
Ailmar, 1
Phærael, 1
Bel-Ithlirel Swiftstrider, 1
Erato, 1
Ariel, 1
Arya, 1
Elpida, 1
Swan Alphon, 1
Heni Hethurin, 1
Kanut Nardhon, 1
The Phantom Wanderer, 0
Bloodsign, 0
Rorthal Xa'zan, 0
Cirerin Stillflow, 0
Slyndis Fendethyl, 0
Divares Tiseran, 0
Harzun 'The Silent Stalker' Krivan, 0
Leroy Vilewing, 0
Aaron Whitestone, 0
Hubris Rakarth, 0
Molostroi Demonheart, 0
Cirgil Elvenslaver, 0
Valyr Dragonrider, 0
Telok Beastslayer, 0
Moreth Ghosthunter, 0
Shorna Coldeyes, 0
Fersi Fellbane, 0
Fekhavi Fireblade, 0
Verbil Amberweb, 0
Darkvenom, 0
Novo Nightshade, 0
Verlin Ravencaller, 0
Ian Silentorc, 0
Merc Priestess 1, 0
Finubar, 0
Tyrielle, 0
Korhil, 

In [27]:
len(perf)

105

In [ ]:
  def standings(self):
    tpkeys = (
        "cas", "casdiff",
        "score", "scorediff",
        "td", "tddiff",
        "pts", "prestige"
    )
    template = {k: 0 for k in tpkeys}
    template["cto"] = -1
    template["hth"] = -1
    template["perf"] = []
    S = collections.defaultdict(lambda: copy.deepcopy(template))
    Ccto = self.config.get("cto", {})
    Chth = self.config.get("hth", {})
    Co = self.config.get("order")
    HTH_results = []
    for Mu in self.matchups:
      if Mu.excluded == "yes":
        continue
      HTH_result = {}
      for teamId, TP in Mu.config["team"].items():
        if teamId.isdecimal():
          Te = cibblbibbl.team.Team(int(teamId))
        else:
          Te = self.get_team(teamId)
        r = TP["r"]
        matchId = (Mu.match.Id if Mu.match else None)
        perf = r, matchId
        d = S[teamId]
        d.setdefault("team", Te)
        d["perf"].append(perf)
        for k in tpkeys:
          d[k] += TP.get(k, 0)
        if r.lower() not in ("b", "f", "x", "-", ""):
          HTH_result[teamId] = TP["td"]
      if len(HTH_result) == 2:
        HTH_results.append(HTH_result)
    bypts = collections.defaultdict(list)
    for teamId, d in S.items():
      bypts[d["pts"]].append(teamId)
    for pts, teamIds in bypts.items():
      if 1 < len(teamIds):
        pts_HTH_results = [{teamId: 0} for teamId in teamIds]
            # this ensures all nodes for HTH calculation
        for r0 in HTH_results:
            # generate edges by passing results of the group
            # members within the group
          r1 = {
              teamId: tds
              for teamId, tds in r0.items()
              if teamId in teamIds
          }
          if 1 < len(r1):
            # I do not want to pass empty dictionaries nor nodes
            # as those were ensured before
            pts_HTH_results.append(r1)
        pts_HTH = pytourney.tie.hth.calculate(pts_HTH_results)
        for teamId, hth_val in pts_HTH.items():
          S[teamId]["hth"] = Chth.get(teamId, hth_val)
    for teamId, cto_val in Ccto.items():
      S[teamId]["cto"] = cto_val
    # determine missing coin toss
    bytie = collections.defaultdict(list)
    keys = ("pts", "hth", "scorediff", "casdiff", "cto")
    for teamId, d in S.items():
      bytie[tuple(d[k] for k in keys)].append(teamId)
    for k, teamIds in bytie.items():
      if 1 < len(teamIds):
        cto_val = -112  # indicate missing
        for teamId in teamIds:
          S[teamId]["cto"] = cto_val
          Ccto = self.config.setdefault("cto", {})
              # this ensures that I write to the config file
          Ccto[teamId] = cto_val
    # determine order
    if Co is None:
      order = sorted(S, key=(
          lambda teamId: (
              -S[teamId]["pts"],
              +S[teamId]["hth"],
              -S[teamId]["tddiff"],
              -S[teamId]["casdiff"],
              -S[teamId]["cto"]
          )
      ))
    else:
      order = [str(teamId) for teamId in Co]
    S.default_factory = None
        # close defaultdict so it can raise KeyError exceptions
    return [S[teamId] for teamId in order if teamId in S]

In [7]:
pprint(T.standings())

[{'cas': 13,
  'casdiff': 7,
  'cto': -1,
  'hth': -1,
  'perf': [('W', 3946742),
           ('W', 3950955),
           ('W', 3951532),
           ('W', 3956796),
           ('W', 3959048)],
  'prestige': 150,
  'pts': 15,
  'score': 16,
  'scorediff': 14,
  'td': 16,
  'tddiff': 14,
  'team': Team(886114)},
 {'cas': 9,
  'casdiff': -1,
  'cto': -1,
  'hth': -1,
  'perf': [('W', 3946651),
           ('W', 3947024),
           ('L', 3951532),
           ('W', 3952784),
           ('W', 3958793)],
  'prestige': 120,
  'pts': 12,
  'score': 9,
  'scorediff': 1,
  'td': 9,
  'tddiff': 1,
  'team': Team(886167)},
 {'cas': 12,
  'casdiff': 2,
  'cto': -1,
  'hth': -1,
  'perf': [('W', 3946296),
           ('W', 3948649),
           ('W', 3951787),
           ('L', 3952784),
           ('L', 3959048)],
  'prestige': 90,
  'pts': 9,
  'score': 10,
  'scorediff': 2,
  'td': 10,
  'tddiff': 2,
  'team': Team(886002)},
 {'cas': 9,
  'casdiff': -5,
  'cto': -1,
  'hth': 1,
  'perf': [('L', 3946651